In [1]:
import numpy as np
import os

import pickle
import random
from tqdm.notebook import tqdm_notebook

In [2]:
data_folder = '4_7_actualq1'

In [3]:
assert data_folder is not None
!rm -r ../data/$data_folder
!gsutil -m cp -r gs://landlord_ai/$data_folder ../data/

rm: cannot remove '../data/4_7_actualq1': No such file or directory
Copying gs://landlord_ai/4_7_actualq1/0.pkl...
Copying gs://landlord_ai/4_7_actualq1/1.pkl...
Copying gs://landlord_ai/4_7_actualq1/2.pkl...
Copying gs://landlord_ai/4_7_actualq1/3.pkl...
Copying gs://landlord_ai/4_7_actualq1/4.pkl...
Copying gs://landlord_ai/4_7_actualq1/5.pkl...
Copying gs://landlord_ai/4_7_actualq1/7.pkl...
Copying gs://landlord_ai/4_7_actualq1/8.pkl...
Copying gs://landlord_ai/4_7_actualq1/6.pkl...
/ [9/9 files][  2.0 GiB/  2.0 GiB] 100% Done                                    
Operation completed over 9 objects/2.0 GiB.                                      


In [4]:
directories = ['../data/' + data_folder]

filenames = []
for directory in directories:
    filenames.extend([directory + '/' + file for file in os.listdir(directory)])

In [5]:
all_history_matrices = []
all_move_vectors = []
all_hand_vectors = []
all_y = []

for filename in tqdm_notebook(filenames):
    with open(filename, 'rb') as f:
        history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
        
    all_history_matrices.extend(history_matrices)
    all_move_vectors.extend(move_vectors)
    all_hand_vectors.extend(hand_vectors)
    all_y.extend(y)

In [6]:
# shuffle all
feature_limit = 70

def densify(sparse_matrix):
    return [x[:feature_limit].todense().astype(np.int8) for x in tqdm_notebook(sparse_matrix)]

all_history_matrices = np.array(densify(all_history_matrices), dtype=np.int8)

In [7]:
all_move_vectors = np.array(all_move_vectors)
all_hand_vectors = np.array(all_hand_vectors)
all_y = np.array(all_y)

In [8]:
from sklearn.model_selection import train_test_split
train_hm, test_hm, train_mv, test_mv, train_hv, test_hv, train_y, test_y = \
    train_test_split(all_history_matrices, all_move_vectors, all_hand_vectors, all_y, test_size=0.1, shuffle=True)

In [9]:
!rm -r ../data/{data_folder}_agg/
!mkdir ../data/{data_folder}_agg/

rm: cannot remove '../data/4_7_actualq1_agg/': No such file or directory


In [11]:
with open('../data/' + data_folder + '_agg/train.pkl', 'wb') as f:
    pickle.dump((train_hm, train_mv, train_hv, train_y), f, protocol=4)

with open('../data/' + data_folder + '_agg/test.pkl', 'wb') as f:
    pickle.dump((test_hm, test_mv, test_hv, test_y), f, protocol=4)

In [12]:
!gsutil cp -r ../data/{data_folder}_agg/ gs://landlord_ai/

Copying file://../data/4_7_actualq1_agg/train.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../data/4_7_actualq1_agg/test.pkl [Content-Type=application/octet-stream]...
\ [2 files][  2.4 GiB/  2.4 GiB]   92.2 MiB/s                                   
Operation completed over 2 objects/2.4 GiB.     